In [1]:
import psycopg2
import pandas as pd
import plotly.express as px
import missingno as msno
import sqlalchemy as sa
import psycopg2
from sqlalchemy import create_engine

In [2]:
path = r'C:\Users\nargi\Documents\coding temple\week 6\sql day 2\Taylor_Train.csv'
df = pd.read_csv(path,encoding='Latin-1')

In [11]:
df

,City,Country,Venue,Opening act(s),Attendance (tickets sold / available),Revenue,Tour
0,Evansville,United States,Roberts Municipal Stadium,Gloriana\r\nKellie Pickler,"7,463 / 7,463","$360,617",Fearless_Tour
1,Jonesboro,United States,Convocation Center,Gloriana\r\nKellie Pickler,"7,822 / 7,822","$340,328",Fearless_Tour
2,St. Louis,United States,Scottrade Center,Gloriana\r\nKellie Pickler,"13,764 / 13,764","$650,420",Fearless_Tour
3,Alexandria,United States,Bishop Ireton High School,Gloriana\r\nKellie Pickler,,,Fearless_Tour
4,North Charleston,United States,North Charleston Coliseum,Gloriana\r\nKellie Pickler,"8,751 / 8,751","$398,154",Fearless_Tour
...,...,...,...,...,...,...,...
440,Sydney,Australia,ANZ Stadium,Charli XCX\r\nBroods,"72,805 / 72,805","$7,686,564",Reputation_Stadium_Tour
441,Brisbane,Australia,The Gabba,Charli XCX\r\nBroods,"43,907 / 43,907","$4,338,127",Reputation_Stadium_Tour
442,Auckland,New Zealand,Mount Smart Stadium,Charli XCX\r\nBroods,"35,749 / 35,749","$3,617,593",Reputation_Stadium_Tour
443,Tokyo,Japan,Tokyo Dome,Charli XCX,"100,109 / 100,109","$14,859,847",Reputation_Stadium_Tour


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   City                                   445 non-null    object
 1   Country                                445 non-null    object
 2   Venue                                  445 non-null    object
 3   Opening act(s)                         444 non-null    object
 4   Attendance (tickets sold / available)  442 non-null    object
 5   Revenue                                442 non-null    object
 6   Tour                                   445 non-null    object
dtypes: object(7)
memory usage: 24.5+ KB


In [13]:
df.isna()

,City,Country,Venue,Opening act(s),Attendance (tickets sold / available),Revenue,Tour
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
440,False,False,False,False,False,False,False
441,False,False,False,False,False,False,False
442,False,False,False,False,False,False,False
443,False,False,False,False,False,False,False


In [4]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
df = df.rename(columns = {'attendance_(tickets_sold_/_available)':'attendance'})
df[['tickets_sold', 'tickets_available']] = df.attendance.str.split("/", expand=True)
del df['attendance']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   city               445 non-null    object
 1   country            445 non-null    object
 2   venue              445 non-null    object
 3   opening_act(s)     444 non-null    object
 4   revenue            442 non-null    object
 5   tour               445 non-null    object
 6   tickets_sold       442 non-null    object
 7   tickets_available  407 non-null    object
dtypes: object(8)
memory usage: 27.9+ KB


In [8]:
df = df.dropna()
df['tickets_available'] = df['tickets_available'].str.replace(',', "")
df['tickets_sold'] = df['tickets_sold'].str.replace(',', "")
df['revenue'] = df['revenue'].str.replace(',', "")

In [9]:
problematic_rows = df[df['revenue'].str.contains('\x97')]
print(problematic_rows)

          city country                             venue opening_act(s)  \
110  Cavendish  Canada  Cavendish Beach Festival Grounds                 

    revenue           tour tickets_sold tickets_available  
110         Fearless_Tour       35000              35000  


In [11]:
# Remove the Chaos
df['revenue'] = df['revenue'].replace('\x97', '0')
df['revenue'] = df['revenue'].str.replace('[\$,]', '', regex=True).astype(int)
df['revenue'].replace('', '0', inplace=True)
df['tickets_sold'].replace('', '0', inplace=True)

# Convert 'revenue' and 'tickets_sold' to integers
df['revenue'] = df['revenue'].astype(int)
df['tickets_sold'] = df['tickets_sold'].astype(int)
df['tickets_available'] = df['tickets_available'].astype(int)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406 entries, 0 to 444
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   city               406 non-null    object
 1   country            406 non-null    object
 2   venue              406 non-null    object
 3   opening_act(s)     406 non-null    object
 4   revenue            406 non-null    int32 
 5   tour               406 non-null    object
 6   tickets_sold       406 non-null    int32 
 7   tickets_available  406 non-null    int32 
dtypes: int32(3), object(5)
memory usage: 23.8+ KB


In [13]:
mean_revenue = df[df['revenue'] != 0]['revenue'].mean()
df['revenue'].replace('0', mean_revenue, inplace=True)

In [15]:
mean_tickets = df[df['tickets_sold'] != 0]['tickets_sold'].mean()
df['tickets_sold'].replace('0', mean_tickets, inplace=True)

In [16]:
df

,city,country,venue,opening_act(s),revenue,tour,tickets_sold,tickets_available
0,Evansville,United States,Roberts Municipal Stadium,Gloriana\r\nKellie Pickler,360617,Fearless_Tour,7463,7463
1,Jonesboro,United States,Convocation Center,Gloriana\r\nKellie Pickler,340328,Fearless_Tour,7822,7822
2,St. Louis,United States,Scottrade Center,Gloriana\r\nKellie Pickler,650420,Fearless_Tour,13764,13764
4,North Charleston,United States,North Charleston Coliseum,Gloriana\r\nKellie Pickler,398154,Fearless_Tour,8751,8751
5,Jacksonville,United States,Jacksonville Veterans Memorial Arena,Gloriana\r\nKellie Pickler,507012,Fearless_Tour,11072,11072
...,...,...,...,...,...,...,...,...
440,Sydney,Australia,ANZ Stadium,Charli XCX\r\nBroods,7686564,Reputation_Stadium_Tour,72805,72805
441,Brisbane,Australia,The Gabba,Charli XCX\r\nBroods,4338127,Reputation_Stadium_Tour,43907,43907
442,Auckland,New Zealand,Mount Smart Stadium,Charli XCX\r\nBroods,3617593,Reputation_Stadium_Tour,35749,35749
443,Tokyo,Japan,Tokyo Dome,Charli XCX,14859847,Reputation_Stadium_Tour,100109,100109
